# Places

- https://developer.here.com/documentation/places/topics/what-is.html

In [ ]:
import requests
from credentials import APP_ID, APP_CODE

lat, lon = 52.5, 13.4
app_id = APP_ID
app_code = APP_CODE
url = (f'https://places.api.here.com/places/v1/discover/search'
       f'?q=Cafe&at={lat},{lon}&app_id={app_id}&app_code={app_code}')
obj = requests.get(url).json()
obj

In [ ]:
for p in [res for res in obj['results']['items']]:
    print('{!r:23} {:4d} m  {}'.format(p['position'], p['distance'], p['title']))

In [ ]:
# from urllib.parse import parse_qs

import requests
from ipyleaflet import Map, GeoJSON, CircleMarker, Marker
from ipyrest.responseviews import ResponseView, zoom_for_bbox

from utils import bbox_for_points

class PlacesView(ResponseView):
    """
    A view that renders HERE places on an ipyleaflet.Map.
    """
    name = 'Places'
    mimetype_pats = mimetype_pats = ['application/json', 'application/vnd\..*\+json']

    def render(self, resp: requests.models.Response) -> Map:
        "Return an ipyleaflet map with the GeoJSON object rendered on it, or None."

        obj = resp.json()
        # lat, lon = list(map(float, parse_qs(resp.request.url)['at'][0].split(',')))
        lat, lon = resp.json()['search']['context']['location']['position']
        bbox = bbox_for_points([p['position'] for p in obj['results']['items']])    
        mins, maxs = bbox
        z = zoom_for_bbox(*(mins + maxs))
        m = Map(center=(lat, lon), zoom=z)
        m += CircleMarker(location=[lat, lon], color='red')
        for p in [res for res in obj['results']['items']]:
            m += Marker(location=p['position'],
                        title='{} ({} m)'.format(p['title'], p['distance']))
        m.add_layer(layer=GeoJSON(data=obj))
        self.data = m
        return m

In [ ]:
import json
from urllib.parse import quote

from ipyrest import Api
from credentials import APP_ID, APP_CODE

url = 'https://places.api.here.com/places/v1/discover/search'
lat, lon = 52.5, 13.4
params = dict(
    q=quote('ATM'),
    at=f'{lat},{lon}',
    app_id=APP_ID,
    app_code=APP_CODE
)
api = Api(url, params=params, additional_views=[PlacesView])
api

In [ ]:
api.url

In [ ]:
obj = api.resp.json()
obj.keys()

In [ ]:
obj = json.loads(api.resp_pane.get_child_named('Content').get_child_named('JSON').value)
obj.keys()

In [ ]:
from ipyleaflet import Map, Marker, CircleMarker
lat, lon = 52.5, 13.4
m = Map(center=[lat, lon], zoom=14)
m += CircleMarker(location=[lat, lon], color='red')
for p in [res for res in obj['results']['items']]:
    m += Marker(location=p['position'],
                title='{} ({} m)'.format(p['title'], p['distance']))
m

## Place Categories

In [ ]:
import json
from urllib.parse import quote

from ipyrest import Api
from credentials import APP_ID, APP_CODE

url = 'https://places.cit.api.here.com/places/v1/categories/places'
lat, lon = 52.5, 13.4
params = dict(
    at=f'{lat},{lon}',
    app_id=APP_ID,
    app_code=APP_CODE
)
api = Api(url, params=params)
api

In [ ]:
obj = api.resp.json()
for id in sorted(res['id'] for res in obj['items']):
    print(id)

## Explore by Category

In [ ]:
import json
from urllib.parse import quote

from ipyrest import Api
from credentials import APP_ID, APP_CODE

url = 'https://places.demo.api.here.com/places/v1/discover/explore'
lat, lon = 52.5, 13.4
params = dict(
    at=f'{lat},{lon}',
    cat='cinema',
    app_id=APP_ID,
    app_code=APP_CODE
)
api = Api(url, params=params, additional_views=[PlacesView])
api

In [ ]:
obj = api.resp.json()
for p in [res for res in obj['results']['items']]:
    name_dist = '{} ({} m)'.format(p['title'], p['distance'])
    print(name_dist)